# NB04: ABMC並列性能評価

## 目的

ABMC (Algebraic Block Multi-Color) 順序付けによるICCGの並列性能を評価する:

1. **スレッドスケーリング**: 1, 2, 4, 8スレッドでの性能変化
2. **Setup vs Solve 分離計測**: IC分解(Setup)と反復法(Solve)の時間内訳
3. **ABMC vs Level-Schedule vs BDDC**: 異なる並列化手法の壁時計時間比較
4. **CG反復コンポーネント分析**: SpMV, Apply, InnerProduct, AXPY の個別計測
5. **Block IC (局所IC)**: ブロック独立IC分解モードの性能評価

## 背景

### IC前処理の並列化の課題

IC(0)前処理の前進・後退代入は本質的に逐次的であり、単純には並列化できない。
ABMC順序付けはこのボトルネックを解消する:

| 手法 | Setup (IC分解) | Apply (前進・後退代入) | 色同期バリア |
|------|---------------|-------------------|------------|
| Level-Schedule | 逐次 | レベルごとに parallel_for | あり（数百レベル） |
| ABMC | 色ごとに parallel_for | 色ごとに parallel_for | あり（数色） |
| Block IC | 全ブロック一斉 parallel_for | 全ブロック一斉 parallel_for | なし |

### ABMCの構造

```
色0: [Block0, Block1, ...] ← parallel_for
色1: [Block5, Block6, ...] ← parallel_for
  :      :                       :
```

- 色は逐次処理（色間依存）
- 同色内のブロックは並列処理（独立）
- ブロック内の行は逐次処理

### ABMCの2つの効果

1. **並列性**: ブロック単位の粗粒度並列化で、Level-Scheduleより効率的なスレッド利用
2. **キャッシュ局所性**: BFSによるブロック化で近接行がまとまり、シングルスレッドでも高速

### Block IC（局所IC分解）

各ブロックが独立にIC分解を行い、ブロック外の接続を無視する。
色同期バリアが不要になり最大並列性を実現するが、前処理品質が劣化しCG反復数が増加する。

In [ ]:
import ngsolve
from ngsolve import *
from netgen.occ import *
import time

from sparsesolv_ngsolve import SparseSolvSolver, BDDCPreconditioner, ICPreconditioner
from ngsolve.krylovspace import CGSolver
print("Setup complete")

## 1. テスト問題: トーラスコイル (HCurl curl-curl)

NB02と同じトーラスコイル問題。HCurl order=2, 約148K DOFs。

In [ ]:
p1 = Pnt(0.6, 0, -0.1)
p2 = Pnt(0.4, 0, -0.1)
p3 = Pnt(0.4, 0,  0.1)
p4 = Pnt(0.6, 0,  0.1)
rect = Wire([Segment(p1, p2), Segment(p2, p3), Segment(p3, p4), Segment(p4, p1)])
coil_shape = Revolve(rect, Axis(Pnt(0,0,0), Vec(0,0,1)), 360)
coil_shape.maxh = 0.05

sphere = Sphere(Pnt(0,0,0), 1.0).bc("outer")
shape = Glue([sphere, coil_shape])
shape.solids[0].mat("air")
shape.solids[1].mat("coil")

mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=0.1))
mesh.Curve(3)
print(f"メッシュ: {mesh.ne} 要素")

eps = 1e-6
fes = HCurl(mesh, order=2, dirichlet="outer", nograds=True)
u, v = fes.TrialFunction(), fes.TestFunction()
a = BilinearForm(curl(u)*curl(v)*dx + eps*u*v*dx)
a.Assemble()

J = CoefficientFunction((1/(0.2*0.2)*y/sqrt(x**2+y**2),
                          -1/(0.2*0.2)*x/sqrt(x**2+y**2), 0))
f = LinearForm(J*v*dx("coil"))
f.Assemble()
mat = a.mat
freedofs = fes.FreeDofs()
print(f"自由度数: {fes.ndof}")

## 2. スレッドスケーリング

ABMC、Block IC、Level-Scheduleの3モードをスレッド数を変えて比較する。

In [ ]:
nruns = 3
thread_counts = [1, 2, 4, 8]

results = []

configs = [
    ("ABMC",     dict(use_abmc=True,  abmc_block_ic=False)),
    ("Block-IC", dict(use_abmc=True,  abmc_block_ic=True)),
    ("LvSched",  dict(use_abmc=False)),
]

for nt in thread_counts:
    SetNumThreads(nt)
    print(f"\n--- {nt} threads ---")

    for method_name, kwargs in configs:
        with TaskManager():
            _ = SparseSolvSolver(mat, "ICCG", freedofs, tol=1e-8, shift=1.5, **kwargs)

        setup_times, solve_times, iters = [], [], 0
        for run in range(nruns):
            with TaskManager():
                t0 = time.perf_counter()
                solver = SparseSolvSolver(mat, "ICCG", freedofs,
                                         abmc_num_colors=8,
                                         tol=1e-8, maxiter=2000, shift=1.5, **kwargs)
                solver.diagonal_scaling = True
                t_setup = time.perf_counter() - t0

                gfu = GridFunction(fes)
                t0 = time.perf_counter()
                res = solver.Solve(f.vec, gfu.vec)
                t_solve = time.perf_counter() - t0
            setup_times.append(t_setup)
            solve_times.append(t_solve)
            iters = res.iterations

        avg_setup = sum(setup_times) / nruns
        avg_solve = sum(solve_times) / nruns
        results.append((nt, method_name, avg_setup * 1000, avg_solve * 1000, iters))
        print(f"  {method_name:>8}: setup={avg_setup*1000:.1f}ms, "
              f"solve={avg_solve*1000:.1f}ms, iters={iters}")

In [ ]:
EQ, DA = chr(61), chr(45)
print(f"\nトーラスコイル HCurl order=2, {fes.ndof} DOFs")
print(f"{EQ*85}")
print(f"{'Threads':>7} | {'Method':>8} | {'Setup(ms)':>9} | {'Solve(ms)':>9} | "
      f"{'Total(ms)':>9} | {'Iters':>5} | {'ms/iter':>7}")
print(f"{DA*85}")
for nt, method, s, v, it in results:
    ms_per = v / it if it > 0 else 0
    print(f"{nt:>7} | {method:>8} | {s:>9.1f} | {v:>9.1f} | {s+v:>9.1f} | {it:>5} | {ms_per:>7.1f}")
print(f"{EQ*85}")

print(f"\n--- Solveスケーリング (1スレッド基準) ---")
for method_name in ["ABMC", "Block-IC", "LvSched"]:
    base = [v for nt, m, _, v, _ in results if m == method_name and nt == 1]
    if not base:
        continue
    bt = base[0]
    for nt, m, _, v, _ in results:
        if m == method_name:
            speedup = bt / v if v > 0 else 0
            print(f"  {method_name:>8} {nt}T: {speedup:.2f}x")

### 観察

- **1スレッドでもABMCが高速**: BFSブロック化によるキャッシュ局所性改善の効果
- **ms/iter**: ABMCが全スレッド数でLevel-Scheduleより高速
- **Block IC**: ms/iterは最速だが、反復数の増加が支配的で総時間はABMCに劣る
- **Setup時間**: 全ケースで1-2msで無視できるレベル。性能差はSolve（反復ごとのApply）で決まる

## 3. ABMC vs BDDC (8スレッド)

ABMCはSetupが軽量、BDDCは反復数が少ない。
問題規模によって最適な手法が異なる。

In [ ]:
SetNumThreads(8)

comparison = []

# ABMC ICCG (8 threads)
setup_times, solve_times = [], []
for run in range(nruns):
    with TaskManager():
        t0 = time.perf_counter()
        solver = SparseSolvSolver(mat, "ICCG", freedofs,
                                 use_abmc=True, abmc_num_colors=8,
                                 tol=1e-8, maxiter=2000, shift=1.5)
        solver.diagonal_scaling = True
        t_setup = time.perf_counter() - t0

        gfu = GridFunction(fes)
        t0 = time.perf_counter()
        res = solver.Solve(f.vec, gfu.vec)
        t_solve = time.perf_counter() - t0
    setup_times.append(t_setup)
    solve_times.append(t_solve)

avg_setup = sum(setup_times) / nruns
avg_solve = sum(solve_times) / nruns
comparison.append(("ICCG+ABMC", res.iterations, avg_setup, avg_solve))
print(f"ICCG+ABMC: {res.iterations} iters, setup={avg_setup:.3f}s, "
      f"solve={avg_solve:.3f}s, total={avg_setup+avg_solve:.3f}s")

# BDDC (8 threads)
setup_times, solve_times = [], []
for run in range(nruns):
    with TaskManager():
        t0 = time.perf_counter()
        bddc = BDDCPreconditioner(a, fes, coarse_inverse="sparsecholesky")
        t_setup = time.perf_counter() - t0

        gfu = GridFunction(fes)
        t0 = time.perf_counter()
        inv = CGSolver(mat=mat, pre=bddc, maxiter=500, tol=1e-8, printrates=False)
        gfu.vec.data = inv * f.vec
        t_solve = time.perf_counter() - t0
    setup_times.append(t_setup)
    solve_times.append(t_solve)

avg_setup = sum(setup_times) / nruns
avg_solve = sum(solve_times) / nruns
comparison.append(("BDDC", inv.iterations, avg_setup, avg_solve))
print(f"BDDC:      {inv.iterations} iters, setup={avg_setup:.3f}s, "
      f"solve={avg_solve:.3f}s, total={avg_setup+avg_solve:.3f}s")

In [ ]:
EQ, DA = chr(61), chr(45)
print(f"\nトーラスコイル HCurl order=2, {fes.ndof} DOFs, 8 threads")
print(f"{EQ*70}")
print(f"{'Method':>12} | {'Iters':>5} | {'Setup(s)':>8} | {'Solve(s)':>8} | "
      f"{'Total(s)':>8} | {'Setup%':>6}")
print(f"{DA*70}")
for name, iters, s, v in comparison:
    pct = s / (s + v) * 100
    print(f"{name:>12} | {iters:>5} | {s:>8.3f} | {v:>8.3f} | {s+v:>8.3f} | {pct:>5.1f}%")
print(f"{EQ*70}")

print(f"\nBDDC Setupコスト: {comparison[1][2]:.3f}s")
print(f"ABMC Setupコスト: {comparison[0][2]*1000:.1f}ms")
print(f"Setupコスト比: BDDC / ABMC = {comparison[1][2]/comparison[0][2]:.0f}x")

## 4. CG反復コンポーネント分析

CG反復1回の構成要素を個別に計測し、並列スケーリングのボトルネックを特定する。

- **SpMV**: 疎行列ベクトル積 `y = A * x`（NGSolve側で実行、メモリ帯域律速）
- **IC Apply**: 前処理適用 `z = M^{-1} * r`（SparseSolv側、3モード比較）
- **InnerProduct / AXPY**: ベクトル演算

In [ ]:
nreps = 500
x_vec = f.vec.CreateVector()
y_vec = f.vec.CreateVector()
x_vec.FV().NumPy()[:] = 1.0

def make_ic(mode):
    pre = ICPreconditioner(mat, freedofs, shift=1.5)
    if mode == "ABMC":
        pre.use_abmc = True
        pre.abmc_num_colors = 8
        pre.diagonal_scaling = True
    elif mode == "Block-IC":
        pre.use_abmc = True
        pre.abmc_num_colors = 8
        pre.abmc_block_ic = True
        pre.diagonal_scaling = True
    else:
        pre.diagonal_scaling = True
    pre.Update()
    return pre

comp_results = []
for nt in [1, 2, 4, 8]:
    SetNumThreads(nt)
    row = {"threads": nt}

    with TaskManager():
        y_vec.data = mat * x_vec
        t0 = time.perf_counter()
        for _ in range(nreps):
            y_vec.data = mat * x_vec
        row["SpMV"] = (time.perf_counter() - t0) / nreps * 1000

    for mode in ["LvSched", "ABMC", "Block-IC"]:
        with TaskManager():
            pre = make_ic(mode)
            y_vec.data = pre * x_vec
            t0 = time.perf_counter()
            for _ in range(nreps):
                y_vec.data = pre * x_vec
            row[f"Apply({mode})"] = (time.perf_counter() - t0) / nreps * 1000

    with TaskManager():
        _ = InnerProduct(x_vec, y_vec)
        t0 = time.perf_counter()
        for _ in range(nreps):
            _ = InnerProduct(x_vec, y_vec)
        row["Dot"] = (time.perf_counter() - t0) / nreps * 1000

    with TaskManager():
        t0 = time.perf_counter()
        for _ in range(nreps):
            y_vec.data += 0.5 * x_vec
        row["AXPY"] = (time.perf_counter() - t0) / nreps * 1000

    comp_results.append(row)

# Print table
EQ, DA = chr(61), chr(45)
print(f"\nCG反復コンポーネント計測 ({nreps} reps, {fes.ndof} DOFs)")
print(f"{EQ*90}")
keys = ["SpMV", "Apply(LvSched)", "Apply(ABMC)", "Apply(Block-IC)", "Dot", "AXPY"]
header = f"{'T':>2} | " + " | ".join(f"{k:>15}" for k in keys)
print(header)
print(f"{DA*90}")
for r in comp_results:
    vals = " | ".join(f"{r[k]:>14.3f}ms" for k in keys)
    print(f"{r['threads']:>2} | {vals}")
print(f"{EQ*90}")

# Scaling
print(f"\n--- Apply スケーリング (1T基準) ---")
for mode in ["LvSched", "ABMC", "Block-IC"]:
    k = f"Apply({mode})"
    base = comp_results[0][k]
    for r in comp_results:
        print(f"  {mode:>8} {r['threads']}T: {base/r[k]:.2f}x  ({r[k]:.1f}ms)")

### 観察

- **SpMV はメモリ帯域律速**: 2Tで~1.15x程度しかスケールしない。これはNGSolve側の演算でSparseSolvでは制御不可
- **ABMC Apply は1TでもLvSchedの2倍速い**: キャッシュ局所性の効果が大きい
- **Block IC Apply が最速**: 色同期バリア不要の効果。ただしCG全体では反復数増加が支配的
- **CG全体のボトルネック**: Apply(~60%) > SpMV(~30%) > ベクトル演算(~10%)
- **2Tでのスケーリング**: Apply は1.5-1.7x出ているが、SpMV の1.15x がCG全体の足を引っ張る

## まとめ

### 性能特性 (8スレッド, 148K DOFs)

| 手法 | CG Solve | 反復数 | ms/iter | Apply単体 | Apply加速(1T比) |
|------|----------|--------|---------|-----------|----------------|
| Level-Schedule | ~3.4s | ~435 | ~7.9 | 5.9ms | 2.6x |
| **ABMC** | **~2.5s** | ~444 | **~5.6** | **2.9ms** | **2.6x** |
| Block IC | ~5.6s | ~1196 | ~4.7 | 2.2ms | 2.4x |
| BDDC | ~1.9s (total) | ~46 | --- | --- | --- |

### ABMC の効果

1. **Apply が高速**: 1TでもLvSchedの2倍速、8Tでも2倍速を維持
2. **キャッシュ局所性**: BFSブロック化の恩恵で、並列化以前にシングルスレッドで大幅改善
3. **Setupコスト**: ~2ms。BDDCの~1100msに比べ無視できるレベル

### Block IC の評価

- Apply単体では最速（色同期バリア不要）
- しかし前処理品質劣化で反復数が~3倍に増加し、CG全体では不利
- メリットが出るのはApply時間が圧倒的に支配的な超大規模問題に限られる

### 並列スケーリングのボトルネック

- CG反復の~30%を占めるSpMVがメモリ帯域律速（NGSolve側）
- SparseSolv側のApplyは2Tで1.6x、8Tで2.6xと良好にスケール
- CG全体のスケーリング改善にはSpMVの高速化が必要（SparseSolv管轄外）

### 適用指針

- **ABMC ICCG**: BDDCのSetupコストが割に合わない中規模問題で最適
- **BDDC**: 大規模問題では反復数の少なさが圧倒的に有利
- **Block IC**: 現状の問題規模では不利。超大規模問題での検証が必要